In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pickle
import os
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from importlib import reload

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../t-recs/')
from trecs.metrics import MSEMeasurement, InteractionSpread, InteractionSpread, InteractionSimilarity, RecSimilarity, RMSEMeasurement, InteractionMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo
import src.globals as globals
import seaborn as sns

from wrapper.models.bubble import BubbleBurster
from src.utils import *
from src.plotting import plot_measurements 
from src.scoring_functions import cosine_sim, entropy, content_fairness
from wrapper.metrics.evaluation_metrics import *

random_state = np.random.seed(42)
plt.style.use("seaborn")

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

globals.initialize()

/var/folders/sm/hcy50x855gvf2b1qwkjstnvh0000gn/T/ipykernel_6428/2732531481.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [2]:
n_attrs=20
max_iter=1000
n_clusters=25

In [3]:
score_fn = None #'content_fairness'
probabilistic = False
globals.ALPHA = 0.2
alpha = globals.ALPHA

# User parameters
drift = 0.05
attention_exp=-0.8

In [4]:
binary_ratings_matrix = load_and_process_movielens(file_path='data/ml-100k/u.data')

In [5]:
# Get user and item representations using NMF
user_representation, item_representation = create_embeddings(binary_ratings_matrix, n_attrs=n_attrs, max_iter=max_iter)

Loaded embeddings.


In [6]:
item_representation.shape

(20, 1682)

In [7]:
# Define topic clusters using NMF
item_cluster_ids, item_cluster_centers = get_clusters(item_representation.T, name='item', n_clusters=n_clusters, n_attrs=n_attrs, max_iter=max_iter)
user_cluster_ids, user_cluster_centers = get_clusters(user_representation, name='user', n_clusters=n_clusters, n_attrs=n_attrs, max_iter=max_iter)

Loaded clusters.
Loaded clusters.


/Users/madisonthantu/miniforge3/envs/fairRS/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/madisonthantu/miniforge3/envs/fairRS/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
num_users = len(user_representation)
num_items = len(item_representation)
print(f'Number of items: {num_items}')
print(f'Number of users: {num_users}')

users = Users(actual_user_profiles=user_representation, 
              repeat_interactions=False, 
              drift=drift,
              attention_exp=attention_exp)

Number of items: 20
Number of users: 943


In [9]:
# Create user_pairs by pairing users only with others that are not in the same cluster
user_item_cluster_mapping = user_topic_mapping(user_representation, item_cluster_centers) # TODO: Remove?
experiment_name = 'users_by_topic'
# Create user_pairs by pairing users only with others that are not in the same cluster
inter_cluster_user_pairs, intra_cluster_user_pairs = create_cluster_user_pairs(user_item_cluster_mapping)

In [10]:
mse = MSEMeasurement()
measurements = [
    InteractionMeasurement(),
    MSEMeasurement(),  
    InteractionSpread(),                
    InteractionSimilarity(pairs=inter_cluster_user_pairs, name='inter_cluster_interaction_similarity'), 
    InteractionSimilarity(pairs=intra_cluster_user_pairs, name='intra_cluster_interaction_similarity'), 
    RecSimilarity(pairs=inter_cluster_user_pairs, name='inter_cluster_rec_similarity'), 
    RecSimilarity(pairs=intra_cluster_user_pairs, name='intra_cluster_rec_similarity'), 
    UserMSEMeasurement(),
    SerendipityMetric(), 
    DiversityMetric(), 
    NoveltyMetric(),
    RecallMeasurement(),
    MeanNumberOfTopics(),
]

In [11]:
# Model
config = {
    'actual_user_representation': users,
    'actual_item_representation': item_representation,
    'item_topics': item_cluster_ids,
    'num_attributes': n_attrs,
    'num_items_per_iter': 10,
    'seed': 42,
    'record_base_state': True,
}

model_name='myopic'
requires_alpha = False

if score_fn:
    if score_fn == 'cosine_sim':
        config['score_fn'] = cosine_sim
        requires_alpha = True
    elif score_fn == 'entropy':
        config['score_fn'] = entropy
        requires_alpha = True
    elif score_fn == 'content_fairness':
        config['score_fn'] = content_fairness        
    else:
        raise Exception('Given score function does not exist.')
    model_name = score_fn

if probabilistic:
    config['probabilistic_recommendations'] = True
    model_name += '_prob'

In [12]:
model = BubbleBurster(**config)

model.add_metrics(*measurements)

In [13]:
# Fair Model
train_timesteps=5
model.startup_and_train(timesteps=train_timesteps)

100%|██████████| 5/5 [00:20<00:00,  4.15s/it]


In [14]:
run_timesteps=20
model.run(timesteps=run_timesteps)

100%|██████████| 20/20 [02:20<00:00,  7.01s/it]


In [15]:
import src
reload(src.utils)
from src.utils import *
    
# Determine file name based on parameter values
parameters = f'_{train_timesteps}trainTimesteps_{run_timesteps}runTimesteps_{n_attrs}nAttrs_{n_clusters}nClusters_{drift}Drift_{attention_exp}AttentionExp'
if requires_alpha:
    parameters += f'_{alpha}Lambda'

# Save actual user preferences
final_preferences_dir = 'artefacts/supplementary/final_preferences/'
file_prefix = f'{model_name}_final_preferences'
final_preferences_path = final_preferences_dir + file_prefix + parameters + '.npy'
np.save(final_preferences_path, model.users.actual_user_profiles.value, allow_pickle=True)

# Save measurements
measurements_dir = f'artefacts/supplementary/measurements/'
file_prefix = f'{model_name}_measurements'

measurements_path = measurements_dir + file_prefix + parameters + '.csv'
# np.set_printoptions(threshold=sys.maxsize)
measurements_df = load_or_create_measurements_df(model, model_name, train_timesteps, measurements_path)
# measurements_df['interaction_histogram'] = measurements_df['interaction_histogram'].tolist()
# saving interaction histogram
path_interaction_histogram = f'{measurements_dir}{model_name}_interaction_histogram{parameters}.csv'
interaction_hist = measurements_df['interaction_histogram'].copy()
interaction_hist[0] = np.repeat(np.nan, interaction_hist[1].shape[0], axis=0)
interaction_hist = np.stack(interaction_hist.values)
test = pd.DataFrame(interaction_hist)
test.to_csv(path_interaction_histogram)
# saving use_mse histogram
path_user_mse_histogram = f'{measurements_dir}{model_name}_user_mse_histogram{parameters}.csv'
user_mse = measurements_df['user_mse'].copy()
user_mse = np.stack(user_mse.values)
test = pd.DataFrame(user_mse)
test.to_csv(path_user_mse_histogram)
# saving all measurements
measurements_df.to_csv(measurements_path)
print('Measurements saved.')

Measurements saved.


In [19]:
def plot_measurements(dfs, parameters_df):
    fig, ax = plt.subplots(4, 3, figsize=(15, 15))
    fig.tight_layout(pad=5.0)

    # plot rec_similarity with timesteps on x axis
    legend_lines, legend_names = [], []
    for i, df in enumerate(dfs):
        ts = df['timesteps']
        name = parameters_df.loc[i, 'model_name']
        if not np.isnan(parameters_df.loc[i, 'Lambda']):
             name += f" (Lambda: {parameters_df.loc[i, 'Lambda']})" 
        legend_names.append(name)
        
        line, = ax[0,0].plot(ts, df['mse'], label=name)
        ax[0,1].plot(ts, df['user_mse'], label=name)
        ax[0,2].plot(ts, df['recall_at_k'], label=name)
    
        if 'interaction_spread' in df.columns:
            ax[1,0].plot(ts, df['interaction_spread'], label=name, alpha=0.5)
        if 'inter_cluster_interaction_similarity' in df.columns:
            ax[1,1].plot(ts, df['inter_cluster_interaction_similarity'], label=name, alpha=0.5)
        if 'intra_cluster_interaction_similarity' in df.columns:
            ax[1,2].plot(ts, df['intra_cluster_interaction_similarity'], label=name, alpha=0.5)

        if 'diversity_metric' in df.columns:
            ax[2,0].plot(ts, df['diversity_metric'], label=name, alpha=0.5)
        if 'inter_cluster_rec_similarity' in df.columns:
            ax[2,1].plot(ts, df['inter_cluster_rec_similarity'], label=name, alpha=0.5)
        if 'intra_cluster_rec_similarity' in df.columns:
            ax[2,2].plot(ts, df['intra_cluster_rec_similarity'], label=name, alpha=0.5)

        if 'serendipity_metric' in df.columns:
            ax[3,0].plot(ts, df['serendipity_metric'], label=name, alpha=0.5)
        if 'novelty_metric' in df.columns:
            ax[3,1].plot(ts, df['novelty_metric'], label=name, alpha=0.5)
        if 'mean_num_topics' in df.columns:
            ax[3,2].plot(ts, df['mean_num_topics'], label=name, alpha=0.5)
        
        legend_lines.append(line)

    for a in ax:
        for b in a:
            b.set_xlabel('Timestep')

    ax[0, 0].set_title('Mean Squared Error')
    ax[0, 0].set_ylabel('MSE')
    
    ax[0, 1].set_title('User Mean Squared Error')
    ax[0, 1].set_ylabel('MSE')
    ax[0, 1].set_xlabel('User ID')
    
    ax[0, 2].set_title('Recall')
    ax[0, 2].set_ylabel('Recall')
    
    ax[1, 0].set_title('Interaction Spread')
    ax[1, 0].set_ylabel('Jaccard Similarity')
    
    ax[1, 1].set_title('Inter Cluster Interaction Similarity')
    ax[1, 1].set_ylabel('Jaccard Similarity')
    
    ax[1, 2].set_title('Intra Cluster Interaction Similarity')
    ax[1, 2].set_ylabel('Jaccard Similarity')
    
    ax[2, 0].set_title('Diversity')
    ax[2, 0].set_ylabel('Diversity')
    
    ax[2, 1].set_title('Inter Cluster Recommendation similarity')
    ax[2, 1].set_ylabel('Jaccard Similarity')
    
    ax[2, 2].set_title('Intra Cluster Recommendation similarity')
    ax[2, 2].set_ylabel('Jaccard Similarity')
    
    ax[3, 0].set_title('Serendipity')
    ax[3, 0].set_ylabel('Serendipity')
    
    ax[3, 1].set_title('Novelty')
    ax[3, 1].set_ylabel('Novelty')

    ax[3, 2].set_title('Mean Number of Topics Interacted per User')
    ax[3, 2].set_ylabel('Mean Number of Topics Interacted per User')
    
    fig.legend(legend_lines, legend_names, loc='upper center', fontsize=14, frameon=False, ncol=5, bbox_to_anchor=(.5, 1.05))

In [66]:
measurements = model.get_measurements()
for i in measurements.keys():
    print(i)

interaction_histogram
mse
interaction_spread
inter_cluster_interaction_similarity
intra_cluster_interaction_similarity
inter_cluster_rec_similarity
intra_cluster_rec_similarity
user_mse
serendipity_metric
diversity_metric
novelty_metric
recall_at_k
mean_num_topics
timesteps
